In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
%%time
## all imports
import polars as pl
import pyarrow.parquet as pq
import dask.dataframe as dd
import os
import shutil
import json
from enum import Enum
from datetime import datetime
from ydata_profiling import ProfileReport
from pathlib import Path
import random

#For excel stuff
import openpyxl
from openpyxl.drawing.image import Image


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Concurrency
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import time

#  Warnings
import warnings
warnings.filterwarnings('ignore')

# Set random state
random_state = 42
# Set figure size
plt.rcParams["figure.figsize"] = (20, 20)


## Set the static file locactions
filepaths = {'vehicles':
    {
    2019: '/kaggle/input/road-accidents-2019-2023/vehicles-2019.csv',
    2020: '/kaggle/input/road-accidents-2019-2023/vehicles-2020.csv',
    2021: '/kaggle/input/road-accidents-2019-2023/vehicles-2021.csv',
    2022: '/kaggle/input/road-accidents-2019-2023/vehicles-2022.csv',
    2023: '/kaggle/input/road-accidents-2019-2023/vehicles-2023.csv',
    },
    'users':{
        2019: '/kaggle/input/road-accidents-2019-2023/users-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/users-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/users-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/users-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/users-2023.csv'
    },
    'places': {
        2019: '/kaggle/input/road-accidents-2019-2023/place-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/place-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/place-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/place-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/place-2023.csv'
    },
    'characteristics':{
        2019: '/kaggle/input/road-accidents-2019-2023/char-2019.csv',
        2020: '/kaggle/input/road-accidents-2019-2023/char-2020.csv',
        2021: '/kaggle/input/road-accidents-2019-2023/char-2021.csv',
        2022: '/kaggle/input/road-accidents-2019-2023/char-2022.csv',
        2023: '/kaggle/input/road-accidents-2019-2023/char-2023.csv'
    }
             
}

class ExtensionMethods:
    @staticmethod
    def generate_filename(filename=None,extension=None):
        current_datetime = datetime.now()
        f = current_datetime.strftime("%Y_%m_%d_%H%M")
        if (filename is None) or (extension is None):
            return str(f)
        else:
            stitched_f = str(filename)+"_"+str(f)+"."+str(extension)
            return str(stitched_f)

    @staticmethod
    def get_file_name_without_extension(filename):
        if filename == None:
            return "Provide a file"
        return Path(filename).stem
        
        

CPU times: user 159 µs, sys: 0 ns, total: 159 µs
Wall time: 164 µs


In [32]:
class CategoryBaseEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def IsCategory(cls):
        return(True if len(cls.__members__)>0 else False)
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {member.name:member.value for member in cls}
    @classmethod
    def to_json(cls, indent=4):
        return json.dumps(cls.to_dict(), indent=indent)

In [33]:
class FileTypeEnum(Enum):
    @classmethod
    def Name(cls):
        return f"{cls.__name__}"
    @classmethod
    def get_description(cls):
        return cls.__doc__ or "No description available"
    @classmethod
    def to_dict(cls):
        return {
            member.name:member.value.to_dict() for member in cls}

In [34]:
%%time
class Vehicles_Enum(FileTypeEnum):
    '''Collection of the categories for file type 'vehicle.csv' '''

    class Num_Acc(CategoryBaseEnum):
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column''' 
    class id_vehicule(CategoryBaseEnum):
        '''The vehicle id  in terms of xxx-xxx'''
    class num_veh(CategoryBaseEnum):
        '''The Number of the vehicle category , something in french '''

    class senc(CategoryBaseEnum): # senc DIRECTION OF TRAFFIC
        '''The Direction of Traffic '''
        UNKNOWN = 0 
        PK_INCREASING = 1 #PK ascending or address ascending
        PK_DECREASING = 2
        NO_REFERENCE_POINT = 3
        RANDOM = -1 ##TODO VERIFY THIS
    
    class catv(CategoryBaseEnum): # catv Vehicle Category
        '''The Category of Vehicle involved in the crash'''
        UNDETERMINED = 0
        BICYCLE = 1
        MOPED_LESS_EQUAL_50CC = 2
        MICROCAR = 3
        TOURISM_VEHICLE = 7
        UTILITY_VEHICLE_PTAC_1_5T_3_5T = 10 #1.5<PTAC<3.5
        HEAVY_TRUCK_PTAC_3_5T_7_5T = 13
        HEAVY_TRUCK_PTAC_OVER_7_5T = 14
        HEAVY_TRUCK_OVER_3_5T_WITH_TRAILER = 15
        TRACTOR_ONLY = 16
        TRACTOR_WITH_SEMI_TRAILER = 17
        SPECIAL_VEHICLE = 20
        AGRICULTURAL_TRACTOR = 21
        SCOOTER_LESS_EQUAL_50CC = 30
        MOTORCYCLE_50CC_125CC = 31
        SCOOTER_50CC_125CC = 32
        MOTORCYCLE_OVER_125CC = 33
        SCOOTER_OVER_125CC = 34
        LIGHT_QUAD_LESS_EQUAL_50CC = 35
        HEAVY_QUAD_OVER_50CC = 36
        BUS = 37
        COACH = 38
        TRAIN = 39
        TRAMWAY = 40
        THREE_WHEELED_VEHICLE_LESS_EQUAL_50CC = 41
        THREE_WHEELED_VEHICLE_50CC_125CC = 42
        THREE_WHEELED_VEHICLE_OVER_125CC = 43
        MOTORIZED_PERSONAL_TRANSPORT = 50
        NON_MOTORIZED_PERSONAL_TRANSPORT = 60
        OTHER_VEHICLE = 99

    class obs(CategoryBaseEnum):  #obs Static ObstacleHit
        ''' The Static/Stationary Obstacle Hit'''
        UNKNOWN = 0
        PARKED_VEHICLE = 1
        TREE_ON_ROADSIDE = 2
        METAL_BARRIER = 3
        CONCRETE_BARRIER = 4
        OTHER_BARRIER=5
        BUILDING_WALL_BRIDGE_PIER = 6 #BUILDING, WALL OR BRIDGE PIER
        VERTICAL_SIGNPOST_OR_EMERGENCY_CALLBOX = 7
        POLE = 8
        URBAN_FURNITURE = 9 # NO CLUE WHAT URBAN FURNITURE
        PARAPET = 10
        REFUGE_ISLAND_BOLLARD = 11 #THE ROAD ISLAND / BOLLARD
        SIDEWALK = 12 #SIDEWALK OR CURB
        DITCH = 13 #DITCH OR EMBANKMENT
        OTHER_OBS_ON_ROADWAY = 14
        OTHER_OBS_ON_SIDEWALK = 15
        ROADWAY_EXIT_WITOUT_OBSTACLES = 16
        AQUEDUCT_HEAD = 17

    class obsm(CategoryBaseEnum): #obsm #Mobile obstacle hit
        '''The Dynamic Obstacle Hit'''
        UNKNOWN = 0
        PEDESTRIAN =1 
        VEHICLE = 2
        RAIL_VEHICLE = 3
        ANIMAL_DOMESTIC = 4
        ANIMAL_WILD = 5
        OTHER = 9
        
    class choc(CategoryBaseEnum): #choc Initial Point of Impact
        '''The Initial Point of Impact of the crash'''
        UNKNOWN = 0
        FRONT = 1
        FRONT_LEFT = 2
        FRONT_RIGHT = 3
        REAR = 4
        REAR_RIGHT = 5
        REAR_LEFT = 6
        SIDE_LEFT = 7
        SIDE_RIGHT = 8
        MULTIPLE = 9

    
    class manv(CategoryBaseEnum): #manv , Main action before crash
        '''The Main Action performed by the user before the crash'''
        UNKNOWN = 0
        CIRCULATING_NO_DIRECTION_CHANGE =1
        CIRCULATING_SAME_DIRECTION = 2 #SAME DIRECTION SAME LANE
        CIRCULATION_BETWEEN_2_LANES = 3
        CIRCULATING_REVERSING = 4
        CIRCULATING_AGAINTS_FLOW_TRAFFIC = 5
        CIRCULATING_CROSSING_MEDIAN_STRIP = 6
        CIRCULATING_IN_BUSLANE_SAME_DIRECTION = 7
        CIRCULATING_IN_BUSLANE_OPP_DIRECTION = 8
        CIRCULATING_INSERTION =9
        CIRCULATING_TURNING_AROUND_CARRIAGE_WAY = 10
        CHANGING_LANE_LEFT = 11
        CHANGING_LANE_RIGHT = 12
        DEPORT_LEFT = 13
        DEPORT_RIGHT = 14
        TURNING_LEFT = 15
        TURNING_RIGHT = 16
        OVERTAKING_LEFT = 17
        OVERTAKING_RIGHT = 18 
        #VARIOUS S
        CROSSING_ROAD = 19
        PARKING_ACTION = 20
        AVOIDANCE_ACTION = 21
        DOOR_OPENED = 22
        STOP_NO_PARKING = 23
        PARKED_WITH_PASS = 24 # PARKED WITH PASSANGERS
        DRIVING_SIDEWALK = 25
        OTHER_ACTIONS = 26
        
    
    class motor(CategoryBaseEnum): # motor
        '''The Type of Motor(In terms of fuel type) involved in the crash'''
        UNKNOWN = 0
        CONVENTIONAL_FUEL = 1 # PETROL, DIESEL ,ETC
        HYBRID_ELECTRIC = 2
        ELECTRIC = 3
        HYDROGEN = 4
        HUMAN_POWERED = 5
        OTHER = 6

    #occ : Number of occupants in the Public transport (bus) including the driver irrespective if they were injured or not
    class occutc(CategoryBaseEnum):
        '''Number of occupants in the Public transport (bus) including the driver irrespective if they were injured or not'''


CPU times: user 2.53 ms, sys: 0 ns, total: 2.53 ms
Wall time: 2.37 ms


In [40]:
class Places_Enum(FileTypeEnum):
    '''Collection of categories for filetype places.csv'''
    class Num_Acc(CategoryBaseEnum):
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column''' 
    class catr(CategoryBaseEnum):
        '''Category of the road'''
        MOTORWAY = 1
        NATIONAL_ROAD = 2
        DEPARTMENTAL_ROAD = 3
        MUNICIPAL_ROAD = 4
        OFF_NETWORK = 5
        PARKING_AREA = 6
        URBAN_METROPOLE_ROAD = 7
        OTHER = 9

    class voie(CategoryBaseEnum):
        '''Number of the road'''

    class v1(CategoryBaseEnum):
        '''Numerical index of the road number (example: 2 bis, 3 ter etc.).'''
    class v2(CategoryBaseEnum):
        '''Alphanumeric index letter of the route.'''

    class circ(CategoryBaseEnum):
        '''Traffic regime:'''
        NOT_PROVIDED = -1
        ONE_WAY = 1
        BIDIRECTIONAL = 2
        SEPARATE_LANES = 3
        VARIABLE_LANE = 4

    class nbv(CategoryBaseEnum):
        '''Total number of traffic lanes.'''
       
    class vosp(CategoryBaseEnum):
        '''Indicates the existence of a reserved lane, regardless of whether or not the accident occurs on this lane.'''
        NOT_PROVIDED = -1
        NOT_APPLICABLE = 0
        CYCLE_TRACK = 1
        CYCLE_LANE = 2
        RESERVED_LANE = 3
        
    class prof(CategoryBaseEnum):
        '''Longitudinal profile describes the slope of the road at the location of the accident:'''
        NOT_PROVIDED = -1
        FLAT = 1
        SLOPE = 2
        CREST = 3
        BOTTOM = 4
        
    class pr(CategoryBaseEnum):
        '''Number of the PR to be attached (number of the upstream terminal). The value -1 means that the PR is not filled in.'''
    
    class pr1(CategoryBaseEnum):
        '''Distance in meters to the PR (in relation to the upstream terminal). The value -1 means that the PR is not entered.'''

    class plan(CategoryBaseEnum):
        '''Plan layout'''
        NOT_PROVIDED = -1
        STRAIGHT_PART = 1
        CURVE_LEFT = 2
        CURVE_RIGHT = 3
        S_SHAPE = 4
        

    class lartpc(CategoryBaseEnum):
        '''Width of the central reservation (CPR) if it exists (in m).'''

    class larrout(CategoryBaseEnum):
        '''Width of the roadway allocated to vehicle traffic excluding emergency lanes, TPCs and parking spaces (in m)'''
        
    class surf(CategoryBaseEnum):
        '''Surface condition'''
        NOT_PROVIDED = -1
        NORMAL = 1
        WET = 2
        PUDDLES = 3
        FLOODED = 4
        SNOWY = 5
        MUDDY = 6
        ICY = 7
        GREASE = 8
        OTHER = 9
       
    class infra(CategoryBaseEnum):
        '''Development - Infrastructure:'''
        NOT_PROVIDED = -1
        NONE = 0
        UNDERGROUND = 1
        BRIDGE = 2
        INTERCHANGE_RAMP = 3
        RAILROAD = 4
        AMENAGED_CROSSROAD = 5
        PEDESTRIAN_ZONE = 6
        TOLL_ZONE = 7
        WORKZONE = 8
        OTHERS = 9
        

    class situ(CategoryBaseEnum):
        '''Situation of the accident'''
        NOT_PROVIDED = -1
        NONE = 0
        ON_ROADWAY = 1
        ON_EMERGENCY_LANE = 2
        ON_SHOULDER = 3
        ON_SIDEWALK = 4
        ON_CYCLE_PATH = 5
        ON_OTHER_SPECIAL_LANE = 6
        OTHERS = 8

    class vma(CategoryBaseEnum):
        '''Maximum speed permitted at the location and time of the accident.'''

In [41]:
class Users_Enum(FileTypeEnum):
    class Num_Acc(CategoryBaseEnum): 
        '''The Index/Number of the Crash follows the pattern yyyyxxxxx and is the index column''' 
    class id_vehicule(CategoryBaseEnum):
        '''The vehicle id  in terms of xxx-xxx'''
        
    class num_veh(CategoryBaseEnum):
        '''Identification of the vehicle taken back for each user occupying this vehicle (including pedestrians who are
attached to the vehicles that hit them)'''

    class place(CategoryBaseEnum): # 
        '''Allows to locate the place occupied in the vehicle by the user at the time of the accident'''
        UNKNOWN = 0 
        DRIVER_SEAT = 1 
        PASSENGER_FIRST_ROW_RIGHT = 2
        PASSENGER_LAST_ROW_RIGHT = 3
        PASSENGER = 4
        PASSENGER_LAST_ROW_MIDDLE = 5
        PASSENGER_FIRST_ROW_MIDDLE = 6
        PASSENGER_MIDDLE_ROW_LEFT = 7
        PASSENGER_MIDDLE_ROW_MIDDLE = 8
        PASSENGER_MIDDLE_ROW_RIGHT = 9
        PEDESTIRIAN = 10
    
    class catu(CategoryBaseEnum): # 
        '''User category'''
        UNDETERMINED = 0
        DRIVER = 1
        PASSENGER = 2
        PEDESTRIAN = 3
        
       
    class grav(CategoryBaseEnum):  
        ''' Severity of the accident: The injured users are classified into three categories of victims plus the uninjured'''
        UNKNOWN = 0
        NO_INJURY = 1
        KILLED = 2
        INJURED_HOSPITALIZED = 3
        MINOR_INJURY = 4
        
       
    class sexe(CategoryBaseEnum):
        '''User's gender'''
        UNKNOWN = 0
        MALE =1 
        FEMALE = 2
        
       
    class an_nais(CategoryBaseEnum): 
        '''Year of birth of the user'''

    class trajet(CategoryBaseEnum):
        '''Reason for traveling at the time of the accident'''
        NOT_PROVIDED = 0  
        HOME_TO_WORK = 1  
        HOME_TO_SCHOOL = 2  
        SHOPPING = 3  
        PROFESSIONAL_USE = 4  
        WALK_OR_LEISURE = 5  
        OTHER = 9  
        
  
    class secu1(CategoryBaseEnum):
        '''The existence of a safety equipment'''
        NOT_PROVIDED = -1  
        NO_EQUIPMENT = 0  
        SEATBELT = 1  
        HELMET = 2  
        CHILD_DEVICE = 3  
        REFLECTIVE_VEST = 4  
        AIRBAG_2W_3W = 5  
        GLOVES_2W_3W = 6  
        GLOVES_AIRBAG_2W_3W = 7  
        NOT_DETERMINABLE = 8  
        OTHER = 9  
            

    class secu2(CategoryBaseEnum):
        '''The existence of a safety equipment'''
        NOT_PROVIDED = -1  
        NO_EQUIPMENT = 0  
        SEATBELT = 1  
        HELMET = 2  
        CHILD_DEVICE = 3  
        REFLECTIVE_VEST = 4  
        AIRBAG_2W_3W = 5  
        GLOVES_2W_3W = 6  
        GLOVES_AIRBAG_2W_3W = 7  
        NOT_DETERMINABLE = 8  
        OTHER = 9  
            
      

    class secu3(CategoryBaseEnum):
        '''The existence of a safety equipment'''
        NOT_PROVIDED = -1  
        NO_EQUIPMENT = 0  
        SEATBELT = 1  
        HELMET = 2  
        CHILD_DEVICE = 3  
        REFLECTIVE_VEST = 4  
        AIRBAG_2W_3W = 5  
        GLOVES_2W_3W = 6  
        GLOVES_AIRBAG_2W_3W = 7  
        NOT_DETERMINABLE = 8  
        OTHER = 9  
            

    class locp(CategoryBaseEnum):
        '''The location of the pedestrian'''
        NOT_PROVIDED = -1  
        NOT_APPLICABLE = 0  
        
        # ON_ROADWAY:  
        OVER_50M_FROM_PEDESTRIAN_CROSSING = 1  
        WITHIN_50M_OF_PEDESTRIAN_CROSSING = 2  
        
        # ON_PEDESTRIAN_CROSSING:  
        WITHOUT_TRAFFIC_LIGHTS = 3  
        WITH_TRAFFIC_LIGHTS = 4  
        
        # VARIOUS:  
        ON_SIDEWALK = 5  
        ON_SHOULDER = 6  
        ON_REFUGE_OR_EMERGENCY_LANE = 7  
        ON_SERVICE_ROAD = 8  
        UNKNOWN = 9  
        
       
    class actp(CategoryBaseEnum):
        '''Action of the pedestrian'''
        NOT_PROVIDED = -1  

        # MOVING:  
        NOT_PROVIDED_OR_NOT_APPLICABLE = 0  
        IN_DIRECTION_OF_COLLIDING_VEHICLE = 1  
        OPPOSITE_DIRECTION_TO_VEHICLE = 2  
        
        # VARIOUS:  
        CROSSING = 3  
        OBSCURED = 4  
        PLAYING_OR_RUNNING = 5  
        WITH_ANIMAL = 6  
        OTHER = 9  
        
        # OTHER_ACTIONS:  
        GETTING_IN_OR_OUT_OF_VEHICLE = 10
        UNKNOWN = 11 


    class etatp(CategoryBaseEnum):
        '''This variable is used to specify whether the injured pedestrian was alone or not'''
        NOT_PROVIDED = -1  
        ALONE = 1  
        ACCOMPANIED = 2  
        IN_GROUP = 3

In [42]:
class Characteristics_Enum(FileTypeEnum):

    class Num_Acc(CategoryBaseEnum):
        '''Accident identifier number'''
       
    class jour(CategoryBaseEnum):
        '''Day of the accident'''
       
    class mois(CategoryBaseEnum):
        '''Month of the accident'''
       
    class an(CategoryBaseEnum):
        '''Year of the accident'''
       
    class hrmn(CategoryBaseEnum):
        '''Hour and minute of the accident'''
       
    class lum(CategoryBaseEnum):
        '''Lighting conditions during the accident'''
        DAYLIGHT = 1
        DUSK_OR_DAWN = 2
        NIGHT_WITHOUT_PUBLIC_LIGHT = 3
        NIGHT_WITH_PUBLIC_LIGHT_OFF = 4
        NIGHT_WITH_PUBLIC_LIGHT_ON = 5
        
       
    class dep(CategoryBaseEnum):
        '''Department code (INSEE code for French departments)'''
       
    class com(CategoryBaseEnum):
        '''Commune code (INSEE code for French communes)'''
       
    class agg(CategoryBaseEnum):
        '''Location of the accident: inside or outside the urban area'''
        OUTSIDE_URBAN_AREA = 1
        INSIDE_URBAN_AREA = 2
        
       
    class int(CategoryBaseEnum):
        '''Type of intersection where the accident occurred'''
        NO_INTERSECTION = 1
        X_INTERSECTION = 2
        T_INTERSECTION = 3
        Y_INTERSECTION = 4
        INTERSECTION_WITH_MORE_THAN_4_BRANCHES = 5
        ROUNDABOUT = 6
        SQUARE = 7
        RAILWAY_CROSSING = 8
        OTHER_INTERSECTION = 9
        
       
    class atm(CategoryBaseEnum):
        '''Atmospheric conditions during the accident'''
        NOT_PROVIDED = -1
        NORMAL = 1
        LIGHT_RAIN = 2
        HEAVY_RAIN = 3
        SNOW_HAIL = 4
        FOG_SMOKE = 5
        STRONG_WIND_STORM = 6
        BLINDING_WEATHER = 7
        CLOUDY_WEATHER = 8
        OTHER = 9
        
       
    class col(CategoryBaseEnum):
        '''Type of collision'''
        NOT_PROVIDED = -1
        TWO_VEHICLES_FRONT = 1
        TWO_VEHICLES_REAR = 2
        TWO_VEHICLES_SIDE = 3
        THREE_VEHICLES_CHAIN = 4
        THREE_VEHICLES_MULTIPLE_COLLISIONS = 5
        OTHER_COLLISION = 6
        NO_COLLISION = 7
        
       
    class adr(CategoryBaseEnum):
        '''Postal address of the accident (provided for accidents inside urban areas)'''
       
    class lat(CategoryBaseEnum):
        '''Latitude of the accident location'''
       
    class long(CategoryBaseEnum):
        '''Longitude of the accident location'''
       

In [49]:
class RoadAccident_Enum(Enum):
    '''This is the meta enum containing filetype enums'''
    VEHICLES = Vehicle_Enum
    USERS = Users_Enum
    PLACES = Places_Enum
    CHARACTERISTICS = Characteristics_Enum
    ## One can access the Enums like a dict now 
    
    @classmethod
    def to_dict(cls):
        return {en.name: en.value.to_dict() for en in cls}
    @classmethod
    def to_json(cls,indent=4):
        return json.dumps(cls.to_dict(), indent=indent)
    @classmethod
    def save_json(cls):
        _data = cls.to_json()
        filepath = os.path.join(os.getcwd(),ExtensionMethods.generate_filename("road_accidents_","json"))
        file = open(filepath, "w")
        file.write(_data)
        file.close()
        print(f"The data has been printed to {filepath}")
        

In [57]:
a = RoadAccident_Enum.save_json()
## To access the enums , we can use it like a big dictionary
b = RoadAccident_Enum.to_dict()
c = b['VEHICLES']['catv']
c

The data has been printed to /kaggle/working/road_accidents__2025_01_12_1552.json


{'UNDETERMINED': 0,
 'BICYCLE': 1,
 'MOPED_LESS_EQUAL_50CC': 2,
 'MICROCAR': 3,
 'TOURISM_VEHICLE': 7,
 'UTILITY_VEHICLE_PTAC_1_5T_3_5T': 10,
 'HEAVY_TRUCK_PTAC_3_5T_7_5T': 13,
 'HEAVY_TRUCK_PTAC_OVER_7_5T': 14,
 'HEAVY_TRUCK_OVER_3_5T_WITH_TRAILER': 15,
 'TRACTOR_ONLY': 16,
 'TRACTOR_WITH_SEMI_TRAILER': 17,
 'SPECIAL_VEHICLE': 20,
 'AGRICULTURAL_TRACTOR': 21,
 'SCOOTER_LESS_EQUAL_50CC': 30,
 'MOTORCYCLE_50CC_125CC': 31,
 'SCOOTER_50CC_125CC': 32,
 'MOTORCYCLE_OVER_125CC': 33,
 'SCOOTER_OVER_125CC': 34,
 'LIGHT_QUAD_LESS_EQUAL_50CC': 35,
 'HEAVY_QUAD_OVER_50CC': 36,
 'BUS': 37,
 'COACH': 38,
 'TRAIN': 39,
 'TRAMWAY': 40,
 'THREE_WHEELED_VEHICLE_LESS_EQUAL_50CC': 41,
 'THREE_WHEELED_VEHICLE_50CC_125CC': 42,
 'THREE_WHEELED_VEHICLE_OVER_125CC': 43,
 'MOTORIZED_PERSONAL_TRANSPORT': 50,
 'NON_MOTORIZED_PERSONAL_TRANSPORT': 60,
 'OTHER_VEHICLE': 99}